In [86]:
# Import packages
import mackinac
import cobra
import pandas as pd
import json
import os
import numpy as np
import seaborn as sns
import medusa
from pickle import load
from termcolor import colored
import matplotlib.pyplot as plt
import pickle
from os.path import join 
import os.path
import itertools
import random
from itertools import chain
import os, json
from optlang.interface import OPTIMAL
import streamlit as st
from cobra.flux_analysis.gapfilling import GapFiller
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.core import DictList
import warnings
warnings.filterwarnings('ignore')
from cobra.util.solver import linear_reaction_coefficients

from medusa.core.ensemble import Ensemble
from medusa.core.feature import Feature
from medusa.core.member import Member
# from probanno import probanno
REACTION_ATTRIBUTES = ['lower_bound', 'upper_bound']
MISSING_ATTRIBUTE_DEFAULT = {'lower_bound':0,'upper_bound':0}

In [87]:
# !pip install watchdog

In [88]:
metabolite_data_dict = {}
base_medium_dict = {}
general_medium = []
base_medium_list = []
path_to_pickle = "/Users/basazinbelhu/probannopy/iterative_pfba/data/medusa_gapfilled_model_pickle/Noise_0.01/"
pkl_files = [pos_pkl for pos_pkl in os.listdir(path_to_pickle) if pos_pkl.endswith('.pkl')]
for filename in pkl_files:
    print(filename)
    with open(path_to_pickle+filename, 'rb') as infile:
        ensemble = pickle.load(infile)
    rxn_id = [m.id for m in ensemble.base_model.metabolites]
    metabolite = [m.name for m in ensemble.base_model.metabolites]
    metabolite_formula = [m.formula for m in ensemble.base_model.metabolites]
    metabolite_data = pd.DataFrame(
    {'rxn_id': [m.id for m in ensemble.base_model.metabolites],
     'formula': [m.formula for m in ensemble.base_model.metabolites],
     'name': [m.name for m in ensemble.base_model.metabolites] })
    for index, row in metabolite_data.iterrows():
        if "C" not in row['formula'] or (row['formula'][0]== "C" and row['formula'][1].islower()):
            metabolite_data.drop(index, inplace=True)
    for r in [i for i in metabolite_data["rxn_id"]]:
        general_medium.append("EX_"+r)
    print(len(general_medium))
    for x in general_medium:
        # check if exists in unique_list or not
        if x not in base_medium_list:
            base_medium_list.append(x)
    metabolite_data_dict[os.path.splitext(filename)[0]]=  metabolite_data
    base_medium_dict[os.path.splitext(filename)[0]] = base_medium_list
    print(len(base_medium_list))

Parabacteroides distasonis ATCC 8503_0.01.pkl
1221
1221
Bifidobacterium longum strain DSM 20219_0.01.pkl
2268
1431
Ruminococcus gnavus ATCC 29149_0.01.pkl
3410
1510
Bacteroides vulgatus ATCC 8482_0.01.pkl
4633
1584
Lactococcus lactis subsp. lactis Il1403_0.01.pkl
5796
1649
Lactobacillus plantarum WCFS1_0.01.pkl
6940
1685
Bacteroides clarus YIT 12056_0.01.pkl
8092
1715
Parabacteroides merdae ATCC 43184_0.01.pkl
9209
1719
Clostridium saccharolyticum WM1_0.01.pkl
10413
1761


In [89]:
def sqrt(x):
    y = []
    for i in range(x):
        y.append(i**i)
    return (y)

In [90]:
import warnings
warnings.filterwarnings("ignore")

In [91]:
data = []
styled_data = []
def eda_func(filepath):
    warnings.filterwarnings("ignore")
    with open(filepath, 'rb') as infile:
        ensemble_cs = pickle.load(infile)
    ensemble_cs
    metabolite_data = pd.DataFrame(
        {'rxn_id': [m.id for m in ensemble_cs.base_model.metabolites],
         'formula': [m.formula for m in ensemble_cs.base_model.metabolites],
         'name': [m.name for m in ensemble_cs.base_model.metabolites] })
    for index, row in metabolite_data.iterrows():
        if "C" not in row['formula'] or (row['formula'][0]== "C" and row['formula'][1].islower()):
            metabolite_data.drop(index, inplace=True)
    print(metabolite_data.head(5))
    general_medium = []
    base_medium_list = []
    for r in [i for i in metabolite_data["rxn_id"]]:
        general_medium.append("EX_"+r)
    for x in general_medium:
            # check if exists in unique_list or not
            if x not in base_medium_list:
                base_medium_list.append(x)
    print(len(base_medium_list))
    len(general_medium)
    ensemble_dict = {}
    for member in ensemble_cs.members:
        cobra_model = ensemble_cs.extract_member(member)
        max_growth = cobra_model.slim_optimize()
        # Assigning new flux to all metabolites 
        ensemble_dict[member.id] = []
        #  carbon_source_value = []
        medium = cobra_model.medium
        for i in medium.keys():
    #         if i in base_medium_list:
            medium[i] = 0.0167      
        cobra_model.medium  = medium  
        true_carbone_source = []
        carbon_source_dict = {}
        carbon_source_value = []
        flux = {}
        # Iterating throught the carbone soure candidates
        for m in base_medium_list:
            if m.endswith("_e"):
                with cobra_model:
                        #update the flux 

                    # Assign a high flux for one carbone sources at a time 
                    #if that carbob source in the medium
                    if m in medium.keys(): 
                        carbon_source_dict[m] = []
                        true_carbone_source.append(m)
                        medium[m] = 100
                        true_carbone_source.append(m)
    #                     print(cobra_model.medium)
                        cobra_model.medium  = medium
                        new_objective_value = cobra_model.optimize().objective_value
                        carbon_source_value.append(new_objective_value)
                        carbon_source_dict[m] = new_objective_value
                        medium[m] = 0.0167
        ensemble_dict[member.id]= carbon_source_dict
    #     print(len(true_carbone_source))
    # import pdfkit
    # pdfkit.from_url("http://google.com", "out.pdf")
    raw_data = pd.DataFrame.from_dict(ensemble_dict,orient='index')
    data.append(raw_data)
    data_1 = raw_data.sub(raw_data.min(axis=1), axis=0)
    styled_data.append(data_1.style.background_gradient(cmap ='viridis', axis = 1).set_properties(**{'font-size': '15px'})) 
    return(styled_data, data)

In [92]:
filepath = "/Users/basazinbelhu/probannopy/iterative_pfba/data/medusa_gapfilled_model_pickle/Noise_0.4/Parabacteroides distasonis ATCC 8503_0.4.pkl"

In [93]:
eda_func(filepath)

       rxn_id      formula                                               name
0  cpd00443_c      C7H6NO2                                               ABEE
1  cpd02920_c   C7H9N5O8P2  2-Amino-4-hydroxy-6-hydroxymethyl-7-8-dihydrop...
4  cpd00683_c   C14H13N6O3                                    Dihydropteroate
5  cpd00114_c  C10H11N4O8P                                                IMP
6  cpd00103_c    C5H9O14P3                                               PRPP
1712


2021-03-22 13:13:06.479 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:06.485 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:06.491 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:06.529 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:06.559 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:06.581 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22

2021-03-22 13:13:10.669 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:10.688 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:10.711 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:10.729 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:10.748 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:10.766 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22

2021-03-22 13:13:16.588 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:16.609 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:16.633 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:16.659 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:16.682 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:16.705 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22

2021-03-22 13:13:20.944 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:20.978 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:21.002 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:21.025 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:21.048 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:13:21.072 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22

...

2021-03-22 13:19:58.668 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:19:58.693 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:19:58.717 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:19:58.741 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:19:58.765 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:19:58.790 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22

2021-03-22 13:20:03.032 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:03.060 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:03.085 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:03.108 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:03.131 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:03.156 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22

2021-03-22 13:20:07.227 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:07.254 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:07.280 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:07.303 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:07.327 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:07.351 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22

2021-03-22 13:20:12.862 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:12.891 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:12.916 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:12.941 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:12.964 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22 13:20:12.995 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-22

([<pandas.io.formats.style.Styler at 0x7fc37b43d040>],
 [                            EX_cpd00106_e  EX_cpd00036_e  EX_cpd00129_e  \
  P. distasonis_gapfilled_0        0.139743       0.139743       0.139743   
  P. distasonis_gapfilled_1        0.088179       0.088179       0.088179   
  P. distasonis_gapfilled_2        0.161369       0.161369       0.161369   
  P. distasonis_gapfilled_3        0.139743       0.139743       0.139743   
  P. distasonis_gapfilled_4        0.146806       0.146806       0.146806   
  ...                                   ...            ...            ...   
  P. distasonis_gapfilled_95       0.273441       0.273441       0.273441   
  P. distasonis_gapfilled_96       0.050826       0.050826       0.050826   
  P. distasonis_gapfilled_97       0.179734       0.179734       0.179734   
  P. distasonis_gapfilled_98       0.161369       0.161369       0.161369   
  P. distasonis_gapfilled_99       0.111286       0.111286       0.111286   
  
                  

In [95]:
dt = data[0].sub(data[0].min(axis=1), axis=0)
dt

,EX_cpd00106_e,EX_cpd00036_e,EX_cpd00129_e,EX_cpd00635_e,EX_cpd00023_e,EX_cpd11595_e,EX_cpd00222_e,EX_cpd00100_e,EX_cpd00154_e,EX_cpd08306_e,...,EX_cpd01080_e,EX_cpd00028_e,EX_cpd11583_e,EX_cpd00264_e,EX_cpd01188_e,EX_cpd00305_e,EX_cpd00367_e,EX_cpd11581_e,EX_cpd00065_e,EX_cpd00156_e
P. distasonis_gapfilled_0,2.613315e-10,0.000000e+00,2.697974e-10,2.697974e-10,2.667495e-10,2.667495e-10,2.667495e-10,2.674027e-10,2.683896e-10,2.683896e-10,...,2.756923e-10,2.756923e-10,2.756923e-10,2.756923e-10,2.756923e-10,2.756923e-10,2.756923e-10,2.756923e-10,2.756923e-10,4.594547e-02
P. distasonis_gapfilled_1,8.189745e-11,8.189745e-11,8.189745e-11,8.189745e-11,1.086996e-10,1.086996e-10,1.086996e-10,3.299028e-13,3.299028e-13,3.298889e-13,...,2.416123e-13,2.416123e-13,2.416123e-13,2.416123e-13,2.416123e-13,2.416123e-13,1.126421e-01,2.968320e-13,2.968320e-13,0.000000e+00
P. distasonis_gapfilled_2,2.400136e-12,2.400136e-12,1.770389e-12,2.213008e-12,1.106448e-12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.906003e-12,1.906003e-12,1.924960e-12,1.858652e-12,1.858652e-12,1.858652e-12,9.535070e-02,6.690148e-12,6.690148e-12,2.291298e-11
P. distasonis_gapfilled_3,8.076873e-14,6.036838e-14,6.036838e-14,6.036838e-14,0.000000e+00,6.036838e-14,6.036838e-14,6.036838e-14,1.493250e-13,1.186162e-12,...,9.331425e-14,9.331425e-14,9.331425e-14,2.282619e-13,9.331425e-14,9.331425e-14,9.331425e-14,9.331425e-14,9.331425e-14,4.594547e-02
P. distasonis_gapfilled_4,1.151940e-12,8.802958e-13,8.802958e-13,8.802958e-13,0.000000e+00,8.231749e-13,1.407152e-12,2.206846e-13,8.222312e-13,4.887979e-12,...,4.880429e-12,4.887979e-12,4.923755e-02,4.031803e-12,4.031803e-12,4.031803e-12,4.031803e-12,4.031803e-12,4.031803e-12,4.923755e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P. distasonis_gapfilled_95,4.938827e-13,4.938827e-13,4.938827e-13,4.938827e-13,4.938827e-13,4.938827e-13,4.938827e-13,4.938827e-13,4.938827e-13,4.938827e-13,...,3.799183e-13,3.799183e-13,3.799183e-13,3.799183e-13,3.799183e-13,3.799183e-13,3.799183e-13,3.799183e-13,3.799183e-13,4.281579e-02
P. distasonis_gapfilled_96,1.895754e-11,1.815787e-11,1.815787e-11,1.815787e-11,1.944381e-11,1.917228e-11,1.917228e-11,1.697129e-12,0.000000e+00,1.310388e-11,...,1.885635e-11,1.885635e-11,1.103116e-02,1.891798e-11,1.891798e-11,1.891798e-11,1.891798e-11,1.891798e-11,1.891798e-11,1.503174e-02
P. distasonis_gapfilled_97,2.270330e-10,2.272665e-10,2.272665e-10,2.272665e-10,1.273281e-10,1.271413e-10,1.277017e-10,0.000000e+00,3.867165e-10,3.867165e-10,...,3.860499e-10,3.860499e-10,6.597756e-02,3.860499e-10,3.860499e-10,3.860499e-10,3.860499e-10,3.860499e-10,3.860499e-10,6.597756e-02
P. distasonis_gapfilled_98,0.000000e+00,1.170682e-10,1.170682e-10,1.183263e-10,1.165996e-10,1.165990e-10,1.165990e-10,1.165990e-10,1.165990e-10,1.165990e-10,...,1.377240e-10,1.377240e-10,1.164442e-10,1.164442e-10,1.164442e-10,1.164442e-10,2.490589e-02,1.164442e-10,1.164442e-10,1.164442e-10


In [115]:
dt= dt.round(5)

In [116]:
dt.index

Index(['P. distasonis_gapfilled_0', 'P. distasonis_gapfilled_1',
       'P. distasonis_gapfilled_2', 'P. distasonis_gapfilled_3',
       'P. distasonis_gapfilled_4', 'P. distasonis_gapfilled_5',
       'P. distasonis_gapfilled_6', 'P. distasonis_gapfilled_7',
       'P. distasonis_gapfilled_8', 'P. distasonis_gapfilled_9',
       'P. distasonis_gapfilled_10', 'P. distasonis_gapfilled_11',
       'P. distasonis_gapfilled_12', 'P. distasonis_gapfilled_13',
       'P. distasonis_gapfilled_14', 'P. distasonis_gapfilled_15',
       'P. distasonis_gapfilled_16', 'P. distasonis_gapfilled_17',
       'P. distasonis_gapfilled_18', 'P. distasonis_gapfilled_19',
       'P. distasonis_gapfilled_20', 'P. distasonis_gapfilled_21',
       'P. distasonis_gapfilled_22', 'P. distasonis_gapfilled_23',
       'P. distasonis_gapfilled_24', 'P. distasonis_gapfilled_25',
       'P. distasonis_gapfilled_26', 'P. distasonis_gapfilled_27',
       'P. distasonis_gapfilled_28', 'P. distasonis_gapfilled_29',
     

In [190]:
# data.loc[0,"EX_cpd00106_e"]

In [118]:
for row in dt.index: 
    print(row)

P. distasonis_gapfilled_0
P. distasonis_gapfilled_1
P. distasonis_gapfilled_2
P. distasonis_gapfilled_3
P. distasonis_gapfilled_4
P. distasonis_gapfilled_5
P. distasonis_gapfilled_6
P. distasonis_gapfilled_7
P. distasonis_gapfilled_8
P. distasonis_gapfilled_9
P. distasonis_gapfilled_10
P. distasonis_gapfilled_11
P. distasonis_gapfilled_12
P. distasonis_gapfilled_13
P. distasonis_gapfilled_14
P. distasonis_gapfilled_15
P. distasonis_gapfilled_16
P. distasonis_gapfilled_17
P. distasonis_gapfilled_18
P. distasonis_gapfilled_19
P. distasonis_gapfilled_20
P. distasonis_gapfilled_21
P. distasonis_gapfilled_22
P. distasonis_gapfilled_23
P. distasonis_gapfilled_24
P. distasonis_gapfilled_25
P. distasonis_gapfilled_26
P. distasonis_gapfilled_27
P. distasonis_gapfilled_28
P. distasonis_gapfilled_29
P. distasonis_gapfilled_30
P. distasonis_gapfilled_31
P. distasonis_gapfilled_32
P. distasonis_gapfilled_33
P. distasonis_gapfilled_34
P. distasonis_gapfilled_35
P. distasonis_gapfilled_36
P. distason

In [119]:
dt

,EX_cpd00106_e,EX_cpd00036_e,EX_cpd00129_e,EX_cpd00635_e,EX_cpd00023_e,EX_cpd11595_e,EX_cpd00222_e,EX_cpd00100_e,EX_cpd00154_e,EX_cpd08306_e,...,EX_cpd01080_e,EX_cpd00028_e,EX_cpd11583_e,EX_cpd00264_e,EX_cpd01188_e,EX_cpd00305_e,EX_cpd00367_e,EX_cpd11581_e,EX_cpd00065_e,EX_cpd00156_e
P. distasonis_gapfilled_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.04595
P. distasonis_gapfilled_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.11264,0.0,0.0,0.00000
P. distasonis_gapfilled_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.09535,0.0,0.0,0.00000
P. distasonis_gapfilled_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.04595
P. distasonis_gapfilled_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.04924,0.0,0.0,0.0,0.00000,0.0,0.0,0.04924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P. distasonis_gapfilled_95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.04282
P. distasonis_gapfilled_96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01103,0.0,0.0,0.0,0.00000,0.0,0.0,0.01503
P. distasonis_gapfilled_97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.06598,0.0,0.0,0.0,0.00000,0.0,0.0,0.06598
P. distasonis_gapfilled_98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.02491,0.0,0.0,0.00000


In [195]:
new_col = []
biomas = []
ensemble = []
rt_dict = {}
met_dict = {}
for rt in dt.index:
    rt_dict[rt] = []
    met_dict[rt]  = []
    for ft in dt.columns:
        if dt.loc[rt, ft]!=0:
            new_col.append(ft)
            biomas.append(dt.loc[rt,ft])
            ensemble.append(rt)
            met_dict[rt].append(ft)
#             print(dt.loc[rt,ft])
#             print(f'row {rt} col {ft}')
            rt_dict[rt].append(dt.loc[rt,ft])

In [196]:
met_dict

{'P. distasonis_gapfilled_0': ['EX_cpd15604_e', 'EX_cpd00156_e'],
 'P. distasonis_gapfilled_1': ['EX_cpd00367_e'],
 'P. distasonis_gapfilled_2': ['EX_cpd00357_e', 'EX_cpd00367_e'],
 'P. distasonis_gapfilled_3': ['EX_cpd15604_e', 'EX_cpd00156_e'],
 'P. distasonis_gapfilled_4': ['EX_cpd11583_e', 'EX_cpd00156_e'],
 'P. distasonis_gapfilled_5': ['EX_cpd00156_e'],
 'P. distasonis_gapfilled_6': ['EX_cpd00156_e'],
 'P. distasonis_gapfilled_7': ['EX_cpd11583_e', 'EX_cpd00156_e'],
 'P. distasonis_gapfilled_8': ['EX_cpd00367_e'],
 'P. distasonis_gapfilled_9': ['EX_cpd00367_e'],
 'P. distasonis_gapfilled_10': ['EX_cpd00039_e'],
 'P. distasonis_gapfilled_11': ['EX_cpd00357_e', 'EX_cpd00367_e'],
 'P. distasonis_gapfilled_12': ['EX_cpd11583_e', 'EX_cpd00156_e'],
 'P. distasonis_gapfilled_13': ['EX_cpd00367_e'],
 'P. distasonis_gapfilled_14': ['EX_cpd00367_e'],
 'P. distasonis_gapfilled_15': ['EX_cpd00039_e', 'EX_cpd00156_e'],
 'P. distasonis_gapfilled_16': ['EX_cpd11583_e', 'EX_cpd00156_e'],
 'P. di

In [192]:
# pd.DataFrame(list(zip(ensemble, biomas)),
#               columns=[new_col])

In [188]:
rt_dict

{'P. distasonis_gapfilled_0': [0.04595, 0.04595],
 'P. distasonis_gapfilled_1': [0.11264],
 'P. distasonis_gapfilled_2': [0.09535, 0.09535],
 'P. distasonis_gapfilled_3': [0.04595, 0.04595],
 'P. distasonis_gapfilled_4': [0.04924, 0.04924],
 'P. distasonis_gapfilled_5': [0.00368],
 'P. distasonis_gapfilled_6': [0.00368],
 'P. distasonis_gapfilled_7': [0.02287, 0.02657],
 'P. distasonis_gapfilled_8': [0.05863],
 'P. distasonis_gapfilled_9': [0.05156],
 'P. distasonis_gapfilled_10': [0.00198],
 'P. distasonis_gapfilled_11': [0.09607, 0.09607],
 'P. distasonis_gapfilled_12': [0.0459, 0.0459],
 'P. distasonis_gapfilled_13': [0.00187],
 'P. distasonis_gapfilled_14': [0.05156],
 'P. distasonis_gapfilled_15': [0.00261, 0.00468],
 'P. distasonis_gapfilled_16': [0.04595, 0.04595],
 'P. distasonis_gapfilled_17': [0.04988, 0.04988],
 'P. distasonis_gapfilled_18': [0.07466, 0.07466],
 'P. distasonis_gapfilled_19': [0.00366],
 'P. distasonis_gapfilled_20': [0.09525, 0.09525],
 'P. distasonis_gapfil

In [189]:
ensemble

['P. distasonis_gapfilled_0',
 'P. distasonis_gapfilled_0',
 'P. distasonis_gapfilled_1',
 'P. distasonis_gapfilled_2',
 'P. distasonis_gapfilled_2',
 'P. distasonis_gapfilled_3',
 'P. distasonis_gapfilled_3',
 'P. distasonis_gapfilled_4',
 'P. distasonis_gapfilled_4',
 'P. distasonis_gapfilled_5',
 'P. distasonis_gapfilled_6',
 'P. distasonis_gapfilled_7',
 'P. distasonis_gapfilled_7',
 'P. distasonis_gapfilled_8',
 'P. distasonis_gapfilled_9',
 'P. distasonis_gapfilled_10',
 'P. distasonis_gapfilled_11',
 'P. distasonis_gapfilled_11',
 'P. distasonis_gapfilled_12',
 'P. distasonis_gapfilled_12',
 'P. distasonis_gapfilled_13',
 'P. distasonis_gapfilled_14',
 'P. distasonis_gapfilled_15',
 'P. distasonis_gapfilled_15',
 'P. distasonis_gapfilled_16',
 'P. distasonis_gapfilled_16',
 'P. distasonis_gapfilled_17',
 'P. distasonis_gapfilled_17',
 'P. distasonis_gapfilled_18',
 'P. distasonis_gapfilled_18',
 'P. distasonis_gapfilled_19',
 'P. distasonis_gapfilled_20',
 'P. distasonis_gapfill

In [179]:
new_col

['EX_cpd15604_e',
 'EX_cpd00156_e',
 'EX_cpd00367_e',
 'EX_cpd00357_e',
 'EX_cpd00367_e',
 'EX_cpd15604_e',
 'EX_cpd00156_e',
 'EX_cpd11583_e',
 'EX_cpd00156_e',
 'EX_cpd00156_e',
 'EX_cpd00156_e',
 'EX_cpd11583_e',
 'EX_cpd00156_e',
 'EX_cpd00367_e',
 'EX_cpd00367_e',
 'EX_cpd00039_e',
 'EX_cpd00357_e',
 'EX_cpd00367_e',
 'EX_cpd11583_e',
 'EX_cpd00156_e',
 'EX_cpd00367_e',
 'EX_cpd00367_e',
 'EX_cpd00039_e',
 'EX_cpd00156_e',
 'EX_cpd11583_e',
 'EX_cpd00156_e',
 'EX_cpd11583_e',
 'EX_cpd00156_e',
 'EX_cpd11583_e',
 'EX_cpd00156_e',
 'EX_cpd00156_e',
 'EX_cpd00357_e',
 'EX_cpd00367_e',
 'EX_cpd15604_e',
 'EX_cpd00156_e',
 'EX_cpd00156_e',
 'EX_cpd00367_e',
 'EX_cpd01188_e',
 'EX_cpd00156_e',
 'EX_cpd11583_e',
 'EX_cpd00156_e',
 'EX_cpd00156_e',
 'EX_cpd00039_e',
 'EX_cpd15604_e',
 'EX_cpd00156_e',
 'EX_cpd11583_e',
 'EX_cpd00156_e',
 'EX_cpd01188_e',
 'EX_cpd00367_e',
 'EX_cpd15604_e',
 'EX_cpd03847_e',
 'EX_cpd00156_e',
 'EX_cpd15604_e',
 'EX_cpd00156_e',
 'EX_cpd00367_e',
 'EX_cpd00

In [149]:
nu = pd.DataFrame.from_dict(rt_dict)
nu.T

,0
P. distasonis_gapfilled_0,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."
P. distasonis_gapfilled_1,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."
P. distasonis_gapfilled_2,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."
P. distasonis_gapfilled_3,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."
P. distasonis_gapfilled_4,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."
...,...
P. distasonis_gapfilled_95,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."
P. distasonis_gapfilled_96,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."
P. distasonis_gapfilled_97,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."
P. distasonis_gapfilled_98,"{'EX_cpd15604_e': 0.04282, 'EX_cpd00156_e': 0...."


In [65]:
data = data[0].round(3)

In [66]:
#Normalize the the data across row
data = data.sub(data.min(axis=1), axis=0)

In [67]:
# select columns with none- zero values
es = data[data!=0].stack()

In [68]:
es.to_dict()

{('P. distasonis_gapfilled_0', 'EX_cpd15604_e'): 0.045999999999999985,
 ('P. distasonis_gapfilled_0', 'EX_cpd00156_e'): 0.045999999999999985,
 ('P. distasonis_gapfilled_1', 'EX_cpd00367_e'): 0.11300000000000002,
 ('P. distasonis_gapfilled_2', 'EX_cpd00357_e'): 0.096,
 ('P. distasonis_gapfilled_2', 'EX_cpd00367_e'): 0.096,
 ('P. distasonis_gapfilled_3', 'EX_cpd15604_e'): 0.045999999999999985,
 ('P. distasonis_gapfilled_3', 'EX_cpd00156_e'): 0.045999999999999985,
 ('P. distasonis_gapfilled_4', 'EX_cpd11583_e'): 0.049000000000000016,
 ('P. distasonis_gapfilled_4', 'EX_cpd00156_e'): 0.049000000000000016,
 ('P. distasonis_gapfilled_5', 'EX_cpd00156_e'): 0.0029999999999999957,
 ('P. distasonis_gapfilled_6', 'EX_cpd00156_e'): 0.0029999999999999957,
 ('P. distasonis_gapfilled_7', 'EX_cpd11583_e'): 0.022999999999999993,
 ('P. distasonis_gapfilled_7', 'EX_cpd00156_e'): 0.026999999999999996,
 ('P. distasonis_gapfilled_8', 'EX_cpd00367_e'): 0.059,
 ('P. distasonis_gapfilled_9', 'EX_cpd00367_e'): 0

In [69]:
cols = data.columns
bt = data.apply(lambda x: x > 0)
rt = bt.apply(lambda x: list(cols[x.values]), axis=1)

In [51]:
rt.to_frame()

,0
P. distasonis_gapfilled_0,"[EX_cpd15604_e, EX_cpd00156_e]"
P. distasonis_gapfilled_1,[EX_cpd00367_e]
P. distasonis_gapfilled_2,"[EX_cpd00357_e, EX_cpd00367_e]"
P. distasonis_gapfilled_3,"[EX_cpd15604_e, EX_cpd00156_e]"
P. distasonis_gapfilled_4,"[EX_cpd11583_e, EX_cpd00156_e]"
...,...
P. distasonis_gapfilled_95,"[EX_cpd15604_e, EX_cpd00156_e]"
P. distasonis_gapfilled_96,"[EX_cpd11583_e, EX_cpd00156_e]"
P. distasonis_gapfilled_97,"[EX_cpd11583_e, EX_cpd00156_e]"
P. distasonis_gapfilled_98,"[EX_cpd00357_e, EX_cpd00367_e]"


In [41]:
list(map(set,data.values))

[{0.0, 0.045999999999999985},
 {0.0, 0.11300000000000002},
 {0.0, 0.096},
 {0.0, 0.045999999999999985},
 {0.0, 0.049000000000000016},
 {0.0, 0.0029999999999999957},
 {0.0, 0.0029999999999999957},
 {0.0, 0.022999999999999993, 0.026999999999999996},
 {0.0, 0.059},
 {0.0, 0.05200000000000002},
 {0.0, 0.0020000000000000018},
 {0.0, 0.096},
 {0.0, 0.045999999999999985},
 {0.0, 0.0020000000000000018},
 {0.0, 0.05200000000000002},
 {0.0, 0.0020000000000000018, 0.0040000000000000036},
 {0.0, 0.045999999999999985},
 {0.0, 0.05000000000000002},
 {0.0, 0.07399999999999998},
 {0.0, 0.003999999999999997},
 {0.0, 0.096},
 {0.0, 0.045999999999999985},
 {0.0, 0.0029999999999999957},
 {0.0, 0.098},
 {0.0, 0.0030000000000000027},
 {0.0, 0.003999999999999997},
 {0.0, 0.044999999999999984},
 {0.0, 0.0029999999999999957},
 {0.0, 0.0020000000000000018},
 {0.0, 0.05499999999999999},
 {0.0, 0.044999999999999984},
 {0.0, 0.6100000000000001},
 {0.0, 0.065},
 {0.0, 0.037000000000000005},
 {0.0, 0.005999999999999

In [70]:
# sns.heatmap(data[0].round(4), annot=True, fmt="d")

In [26]:
styled_data = []
raw_data = []
def data_val(path_to_pickle):
    pkl_files = [pos_pkl for pos_pkl in os.listdir(path_to_pickle) if pos_pkl.endswith('.pkl')]
    for filename in pkl_files:
        print(filename)
        with open(path_to_pickle+filename, 'rb') as infile:
            ensemble_cs = pickle.load(infile)
        ensemble_cs
        metabolite_data = pd.DataFrame(
            {'rxn_id': [m.id for m in ensemble_cs.base_model.metabolites],
             'formula': [m.formula for m in ensemble_cs.base_model.metabolites],
             'name': [m.name for m in ensemble_cs.base_model.metabolites] })
        for index, row in metabolite_data.iterrows():
            if "C" not in row['formula'] or (row['formula'][0]== "C" and row['formula'][1].islower()):
                metabolite_data.drop(index, inplace=True)
        print(metabolite_data.head(5))
        general_medium = []
        base_medium_list = []
        for r in [i for i in metabolite_data["rxn_id"]]:
            general_medium.append("EX_"+r)
        for x in general_medium:
                # check if exists in unique_list or not
                if x not in base_medium_list:
                    base_medium_list.append(x)
        print(len(base_medium_list))
        len(general_medium)
        ensemble_dict = {}
        for member in ensemble_cs.members:
            cobra_model = ensemble_cs.extract_member(member)
            max_growth = cobra_model.slim_optimize()
            # Assigning new flux to all metabolites 
            ensemble_dict[member.id] = []
            #  carbon_source_value = []
            medium = cobra_model.medium
            for i in medium.keys():
        #         if i in base_medium_list:
                medium[i] = 1/len(base_medium_list)      
            cobra_model.medium  = medium  
            true_carbone_source = []
            carbon_source_dict = {}
            carbon_source_value = []
            flux = {}
            # Iterating throught the carbone soure candidates
            for m in base_medium_list:
                if m.endswith("_e"):
                    with cobra_model:
                            #update the flux 

                        # Assign a high flux for one carbone sources at a time 
                        #if that carbob source in the medium
                        if m in medium.keys(): 
                            carbon_source_dict[m] = []
                            true_carbone_source.append(m)
                            medium[m] = 100
                            true_carbone_source.append(m)
        #                     print(cobra_model.medium)
                            cobra_model.medium  = medium
                            new_objective_value = cobra_model.optimize().objective_value
                            carbon_source_value.append(new_objective_value)
                            carbon_source_dict[m] = new_objective_value
                            medium[m] = 1/len(base_medium_list)
            ensemble_dict[member.id]= carbon_source_dict
        #     print(len(true_carbone_source))
        # import pdfkit
        # pdfkit.from_url("http://google.com", "out.pdf")
        data_1 = pd.DataFrame.from_dict(ensemble_dict,orient='index')
        data = data_1.sub(data_1.min(axis=1), axis=0)
        raw_data.append(data)
        styled_data.append(data.style.background_gradient(cmap ='viridis', axis=1).set_properties(**{'font-size': '15px'}))
    return(styled_data, raw_data)

In [27]:
path_to_pickle = "/Users/basazinbelhu/probannopy/iterative_pfba/data/medusa_gapfilled_model_pickle/Train_data/"
pkl_files = [pos_pkl for pos_pkl in os.listdir(path_to_pickle) if pos_pkl.endswith('.pkl')]

In [28]:
data_val(path_to_pickle)

Bacteroides vulgatus ATCC 8482.pkl
       rxn_id      formula                                               name
0  cpd00443_c      C7H6NO2                                               ABEE
1  cpd02920_c   C7H9N5O8P2  2-Amino-4-hydroxy-6-hydroxymethyl-7-8-dihydrop...
4  cpd00683_c   C14H13N6O3                                    Dihydropteroate
5  cpd00114_c  C10H11N4O8P                                                IMP
6  cpd00103_c    C5H9O14P3                                               PRPP
1351


2021-03-19 14:22:01.229 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:01.235 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:01.242 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:01.275 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:01.296 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:01.319 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19

2021-03-19 14:22:03.525 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:03.555 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:03.587 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:03.621 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:03.649 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:03.677 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19

2021-03-19 14:22:05.326 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:05.345 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:05.363 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:05.394 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:05.413 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:05.431 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19

2021-03-19 14:22:06.960 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:06.977 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:07.691 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:07.697 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:07.703 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:22:07.734 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19

...

2021-03-19 14:24:37.426 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:37.443 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:37.461 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:37.480 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:37.499 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:37.517 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19

2021-03-19 14:24:39.086 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:39.108 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:39.130 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:39.151 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:39.171 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:39.193 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19

2021-03-19 14:24:40.858 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:40.879 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:40.901 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:40.921 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:40.943 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:40.974 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19

2021-03-19 14:24:42.585 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:42.615 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:42.635 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:42.653 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:43.356 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19 14:24:43.362 INFO    cobra.medium.boundary_types: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-19

([<pandas.io.formats.style.Styler at 0x7fe793b44a90>,
 [                          EX_cpd00635_e  EX_cpd00023_e  EX_cpd00224_e  \
  B. vulgatus_gapfilled_0    1.141241e-11   1.162276e-11   1.236960e-11   
  B. vulgatus_gapfilled_1    1.375759e-11   1.375759e-11   1.375759e-11   
  B. vulgatus_gapfilled_2    1.736441e-11   1.736441e-11   8.445890e-12   
  B. vulgatus_gapfilled_3    3.231546e-10   3.231546e-10   2.332448e-10   
  B. vulgatus_gapfilled_4    1.151036e-10   1.151036e-10   3.414621e-10   
  B. vulgatus_gapfilled_5    1.880021e-10   1.819804e-10   1.886668e-10   
  B. vulgatus_gapfilled_6    0.000000e+00   3.477843e-13   3.477843e-13   
  B. vulgatus_gapfilled_7    1.801596e-10   1.494626e-10   7.771855e-11   
  B. vulgatus_gapfilled_8    1.180636e-11   1.221834e-11   1.103784e-11   
  B. vulgatus_gapfilled_9    0.000000e+00   2.070913e-14   2.070913e-14   
  B. vulgatus_gapfilled_10   4.304371e-11   4.304371e-11   4.304371e-11   
  B. vulgatus_gapfilled_11   0.000000e+00   1.

In [30]:
styled_data[3]

,EX_cpd00182_e,EX_cpd00073_e,EX_cpd00412_e,EX_cpd00635_e,EX_cpd00246_e,EX_cpd00367_e,EX_cpd00080_e,EX_cpd00184_e,EX_cpd11595_e,EX_cpd00438_e,EX_cpd00249_e,EX_cpd00027_e,EX_cpd00654_e,EX_cpd01171_e,EX_cpd00154_e,EX_cpd08306_e,EX_cpd00277_e,EX_cpd00164_e,EX_cpd00309_e,EX_cpd03279_e,EX_cpd00280_e,EX_cpd00108_e,EX_cpd00540_e,EX_cpd00311_e,EX_cpd11597_e,EX_cpd08305_e,EX_cpd00396_e,EX_cpd00039_e,EX_cpd00011_e,EX_cpd01188_e,EX_cpd00028_e,EX_cpd00069_e,EX_cpd00010_e,EX_cpd00136_e,EX_cpd00060_e,EX_cpd03422_e,EX_cpd03847_e,EX_cpd00066_e,EX_cpd00111_e,EX_cpd01080_e,EX_cpd00322_e,EX_cpd11606_e,EX_cpd00305_e,EX_cpd00264_e
B. clarus_gapfilled_0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007248,0.000000,0.000000,0.000000
B. clarus_gapfilled_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003929,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. clarus_gapfilled_2,0.000000,0.000000,0.006107,0.000000,0.000000,0.006107,0.000000,0.000000,0.000000,0.000000,0.006107,0.000000,0.006107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. clarus_gapfilled_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002813,0.000000,0.000000,0.000000
B. clarus_gapfilled_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003328,0.000000,0.000000,0.000000
B. clarus_gapfilled_5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005678,0.000000,0.000000,0.000000,0.000000
B. clarus_gapfilled_6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007074,0.000000,0.000000,0.000000
B. clarus_gapfilled_7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001902,0.000000,0.000000,0.000000
B. clarus_gapfilled_8,0.000000,0.000000

In [29]:
raw_data[2]

,EX_cpd11576_e,EX_cpd01030_e,EX_cpd00129_e,EX_cpd00276_e,EX_cpd00082_e,EX_cpd00080_e,EX_cpd00224_e,EX_cpd00314_e,EX_cpd00179_e,EX_cpd00100_e,...,EX_cpd00084_e,EX_cpd00264_e,EX_cpd00039_e,EX_cpd00010_e,EX_cpd01080_e,EX_cpd03847_e,EX_cpd00393_e,EX_cpd00119_e,EX_cpd00136_e,EX_cpd00322_e
B. longum subsp. infantis_gapfilled_0,3.200881e-12,3.096062e-12,3.096062e-12,6.721512e-12,3.765668e-13,5.106159e-14,4.786622e-14,4.786622e-14,8.935387e-14,0.000000e+00,...,2.531754e-12,2.531754e-12,2.531754e-12,2.531754e-12,2.531754e-12,3.135995e-12,2.531754e-12,2.531754e-12,2.531754e-12,2.165614e-02
B. longum subsp. infantis_gapfilled_1,1.137995e-11,1.140161e-11,1.140161e-11,1.144493e-11,1.155322e-11,1.159654e-11,4.235709e-13,4.235709e-13,1.349532e-12,4.235709e-13,...,1.170388e-11,1.170388e-11,1.170388e-11,1.170388e-11,1.170388e-11,1.170388e-11,1.170388e-11,1.170388e-11,1.170388e-11,1.262000e-11
B. longum subsp. infantis_gapfilled_2,0.000000e+00,0.000000e+00,0.000000e+00,4.533039e-12,4.533039e-12,4.533040e-12,4.533040e-12,4.533040e-12,4.533040e-12,4.533040e-12,...,4.533037e-12,4.533037e-12,4.533037e-12,4.533037e-12,4.533037e-12,1.532251e-02,4.568201e-12,4.568201e-12,4.568201e-12,2.190949e-04
B. longum subsp. infantis_gapfilled_3,1.337835e-11,1.337835e-11,1.337835e-11,0.000000e+00,4.205054e-10,4.172771e-10,4.172784e-10,4.172784e-10,4.209530e-10,4.208155e-10,...,4.191366e-10,4.191366e-10,4.191366e-10,4.191366e-10,4.191366e-10,4.191366e-10,4.191366e-10,4.191366e-10,4.191366e-10,2.165614e-02
B. longum subsp. infantis_gapfilled_4,1.573043e-10,1.610531e-10,1.610531e-10,1.210299e-10,1.519129e-10,1.628504e-10,1.628504e-10,1.628504e-10,8.416999e-11,9.651700e-12,...,1.679052e-10,1.679052e-10,1.679052e-10,1.679052e-10,8.852968e-11,1.679052e-10,1.679052e-10,1.679052e-10,1.679052e-10,1.322420e-02
B. longum subsp. infantis_gapfilled_5,7.642158e-12,8.008594e-12,2.032069e-11,1.409135e-11,7.891139e-12,1.097780e-11,9.087689e-12,9.087689e-12,9.258740e-12,8.500117e-12,...,8.303962e-12,8.303962e-12,8.303962e-12,8.303962e-12,8.303962e-12,8.163990e-12,4.888610e-12,4.888610e-12,4.888610e-12,3.982595e-11
B. longum subsp. infantis_gapfilled_6,1.680785e-11,1.680785e-11,1.680785e-11,9.595030e-12,0.000000e+00,1.488884e-11,1.641081e-11,1.641081e-11,1.840903e-11,1.832084e-11,...,1.684570e-11,1.684570e-11,1.684570e-11,1.684570e-11,1.684570e-11,1.213565e-02,1.671259e-11,1.671259e-11,1.671259e-11,1.599590e-04
B. longum subsp. infantis_gapfilled_7,3.927108e-11,3.797121e-11,3.671436e-11,3.008809e-11,1.241279e-11,0.000000e+00,0.000000e+00,0.000000e+00,2.952405e-11,2.959632e-11,...,2.952403e-11,2.952403e-11,2.952403e-11,2.952403e-11,2.952363e-11,2.952419e-11,2.952403e-11,2.952403e-11,2.952403e-11,2.165860e-02
B. longum subsp. infantis_gapfilled_8,5.161267e-11,5.143869e-11,5.143869e-11,5.139923e-11,4.875377e-11,4.555120e-11,4.555120e-11,4.555120e-11,3.091693e-11,2.066307e-12,...,4.534572e-11,4.534572e-11,4.534572e-11,4.534572e-11,4.537823e-11,4.532231e-11,4.532231e-11,4.532231e-11,4.532231e-11,2.165614e-02
B. longum subsp. infantis_gapfilled_9,7.564609e-13,7.564609e-13,7.564609e-13,2.630125e-12,5.892418e-12,4.461449e-12,5.350696e-12,5.350696e-12,5.347816e-12,5.064282e-12,...,5.237685e-13,5.237685e-13,5.237685e-13,5.237685e-13,6.069797e-13,7.773639e-05,0.000000e+00,0.000000e+00,0.000000e+00,1.333832e-03


In [10]:
# path_to_pickle = "/Users/basazinbelhu/probannopy/iterative_pfba/data/medusa_gapfilled_model_pickle/Noise_0.2/"
# pkl_files = [pos_pkl for pos_pkl in os.listdir(path_to_pickle) if pos_pkl.endswith('.pkl')]

In [ ]:
# data_val(path_to_pickle)

In [ ]:
styled_data[8]

In [9]:
path_to_pickle = "/Users/basazinbelhu/probannopy/iterative_pfba/data/medusa_gapfilled_model_pickle/New_N/"
pkl_files = [pos_pkl for pos_pkl in os.listdir(path_to_pickle) if pos_pkl.endswith('.pkl')]


In [138]:
def data_valfunc(pickle_file_path, filename):
    with open(pickle_file_path+filename, 'rb') as infile:
        ensemble_cs = pickle.load(infile)
        ensemble_cs
        metabolite_data = pd.DataFrame(
            {'rxn_id': [m.id for m in ensemble_cs.base_model.metabolites],
             'formula': [m.formula for m in ensemble_cs.base_model.metabolites],
             'name': [m.name for m in ensemble_cs.base_model.metabolites] })
        for index, row in metabolite_data.iterrows():
            if "C" not in row['formula'] or (row['formula'][0]== "C" and row['formula'][1].islower()):
                metabolite_data.drop(index, inplace=True)
    #     print(metabolite_data.head(5))
        general_medium = []
        base_medium_list = []
        for r in [i for i in metabolite_data["rxn_id"]]:
            general_medium.append("EX_"+r)
        for x in general_medium:
                # check if exists in unique_list or not
                if x not in base_medium_list:
                    base_medium_list.append(x)
    #     print(len(base_medium_list))
    #     len(general_medium)
        ensemble_dict = {}
        for member in ensemble_cs.members:
            cobra_model = ensemble_cs.extract_member(member)
            max_growth = cobra_model.slim_optimize()
            # Assigning new flux to all metabolites 
            ensemble_dict[member.id] = []
            #  carbon_source_value = []
            medium = cobra_model.medium
            for i in medium.keys():
        #         if i in base_medium_list:
                medium[i] = 1/len(base_medium_list)      
            cobra_model.medium  = medium  
            true_carbone_source = []
            carbon_source_dict = {}
            carbon_source_value = []
            flux = {}
            # Iterating throught the carbone soure candidates
            for m in base_medium_list:
                if m.endswith("_e"):
                    with cobra_model:
                            #update the flux 

                        # Assign a high flux for one carbone sources at a time 
                        #if that carbob source in the medium
                        if m in medium.keys(): 
                            carbon_source_dict[m] = []
                            true_carbone_source.append(m)
                            medium[m] = 100
                            true_carbone_source.append(m)
        #                     print(cobra_model.medium)
                            cobra_model.medium  = medium
                            new_objective_value = cobra_model.optimize().objective_value
                            carbon_source_value.append(new_objective_value)
                            carbon_source_dict[m] = new_objective_value
                            medium[m] = 1/len(base_medium_list)
            ensemble_dict[member.id]= carbon_source_dict
        #     print(len(true_carbone_source))
        # import pdfkit
        # pdfkit.from_url("http://google.com", "out.pdf")
            data_1 = pd.DataFrame.from_dict(ensemble_dict,orient='index')
        data_1 = data_1.sub(data_1.min(axis=1), axis=0)
    return(data_1.style.background_gradient(cmap ='viridis', axis=1).set_properties(**{'font-size': '15px'})) 

In [139]:
for filename in pkl_files:
    data_valfunc(path_to_pickle, filename)

In [16]:
path = "/Users/basazinbelhu/probannopy/iterative_pfba/data/medusa_gapfilled_model_pickle/Noise_0.01csv/"

In [156]:
dt = pd.read_csv(path +"Lactococcus lactis subsp. lactis Il1403_0.01.pkl.csv")

In [157]:
dt

,Unnamed: 0,Parabacteroides distasonis ATCC 8503_0.01.pkl,Bifidobacterium longum strain DSM 20219_0.01.pkl,Ruminococcus gnavus ATCC 29149_0.01.pkl,Bacteroides vulgatus ATCC 8482_0.01.pkl,Lactococcus lactis subsp. lactis Il1403_0.01.pkl
0,P. distasonis_gapfilled_0,"{'EX_cpd00635_e': 0.020546098726918142, 'EX_cp...",NaN,NaN,NaN,NaN
1,P. distasonis_gapfilled_1,"{'EX_cpd00635_e': 0.04109219745379554, 'EX_cpd...",NaN,NaN,NaN,NaN
2,P. distasonis_gapfilled_2,"{'EX_cpd00635_e': 0.0410921974536216, 'EX_cpd0...",NaN,NaN,NaN,NaN
3,P. distasonis_gapfilled_3,"{'EX_cpd00635_e': 0.041092197453795574, 'EX_cp...",NaN,NaN,NaN,NaN
4,P. distasonis_gapfilled_4,"{'EX_cpd00635_e': 0.04109219745379558, 'EX_cpd...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
495,L. lactis_gapfilled_95,NaN,NaN,NaN,NaN,"{'EX_cpd00023_e': 0.0059440256724, 'EX_cpd0002..."
496,L. lactis_gapfilled_96,NaN,NaN,NaN,NaN,"{'EX_cpd00023_e': 0.041507215850306284, 'EX_cp..."
497,L. lactis_gapfilled_97,NaN,NaN,NaN,NaN,"{'EX_cpd00023_e': 0.006257753145235209, 'EX_cp..."
498,L. lactis_gapfilled_98,NaN,NaN,NaN,NaN,"{'EX_cpd00023_e': 0.006248155294555164, 'EX_cp..."


In [17]:
# dt.columns
dt = pd.read_csv(path +"Parabacteroides distasonis ATCC 8503_0.01.pkl.csv")

In [19]:
dt

,Unnamed: 0,Parabacteroides distasonis ATCC 8503_0.01.pkl
0,P. distasonis_gapfilled_0,"{'EX_cpd00635_e': 0.020546098726918142, 'EX_cp..."
1,P. distasonis_gapfilled_1,"{'EX_cpd00635_e': 0.04109219745379554, 'EX_cpd..."
2,P. distasonis_gapfilled_10,"{'EX_cpd00635_e': 0.04109219745379451, 'EX_cpd..."
3,P. distasonis_gapfilled_11,"{'EX_cpd00635_e': 0.04109219745374625, 'EX_cpd..."
4,P. distasonis_gapfilled_12,"{'EX_cpd00635_e': 0.04109219745379554, 'EX_cpd..."
...,...,...
95,P. distasonis_gapfilled_95,"{'EX_cpd00635_e': 0.04109219745379557, 'EX_cpd..."
96,P. distasonis_gapfilled_96,"{'EX_cpd00635_e': 0.041092197453795574, 'EX_cp..."
97,P. distasonis_gapfilled_97,"{'EX_cpd00635_e': 0.02054609872657992, 'EX_cpd..."
98,P. distasonis_gapfilled_98,"{'EX_cpd00635_e': 0.04109219745379558, 'EX_cpd..."


In [128]:
dt['Lactobacillus plantarum WCFS1_0.4.pkl'][110]

"{'EX_cpd00023_e': 0.06735978982036099, 'EX_cpd00224_e': 0.06735978982324364, 'EX_cpd00027_e': 0.06735978981742016, 'EX_cpd01171_e': 0.06735978981753671, 'EX_cpd00154_e': 0.06735978981753671, 'EX_cpd08306_e': 0.06735978981753671, 'EX_cpd00309_e': 0.06735978981738686, 'EX_cpd00108_e': 0.06735978981753671, 'EX_cpd00159_e': 0.06735978981824661, 'EX_cpd00540_e': 0.06735978981905362, 'EX_cpd11597_e': 0.06735978981753671, 'EX_cpd08305_e': 0.06735978981753671, 'EX_cpd00039_e': 0.07265330643716524, 'EX_cpd00011_e': 0.06735978981802222, 'EX_cpd00322_e': 0.06735978981805818, 'EX_cpd00111_e': 0.06735978981802222, 'EX_cpd00028_e': 0.06735978981802222, 'EX_cpd00066_e': 0.06735978981802222, 'EX_cpd01188_e': 0.06735978981802222, 'EX_cpd01080_e': 0.06735978981802222, 'EX_cpd00010_e': 0.06735978981802222, 'EX_cpd00264_e': 0.06735978981802222, 'EX_cpd11606_e': 0.06735978981802222, 'EX_cpd00069_e': 0.06735978981802222, 'EX_cpd00060_e': 0.06735978981802222, 'EX_cpd00305_e': 0.06735978981802222, 'EX_cpd000

In [121]:
dt["Bacteroides vulgatus ATCC 8482_0.4.pkl"][99]

"{'EX_cpd00635_e': 0.6536739717390511, 'EX_cpd00023_e': 0.6536739717438249, 'EX_cpd00224_e': 0.6536739717441643, 'EX_cpd11595_e': 0.6536739717390511, 'EX_cpd01217_e': 0.6536739717441646, 'EX_cpd00027_e': 0.6536739717407711, 'EX_cpd01171_e': 0.6536739717390511, 'EX_cpd00154_e': 0.6536739717390511, 'EX_cpd08306_e': 0.6536739717390511, 'EX_cpd00164_e': 0.6536739717390511, 'EX_cpd00309_e': 0.65367397173957, 'EX_cpd00280_e': 0.6536739717398535, 'EX_cpd00108_e': 0.6536739717398535, 'EX_cpd00159_e': 0.6536739717441661, 'EX_cpd00540_e': 0.6536739717398535, 'EX_cpd11597_e': 0.6536739717398535, 'EX_cpd08305_e': 0.6536739717398535, 'EX_cpd00396_e': 0.6536739717398535, 'EX_cpd00039_e': 0.6536739717398535, 'EX_cpd00011_e': 0.6536739717398535, 'EX_cpd00322_e': 0.9016601964505422, 'EX_cpd00111_e': 0.6536739717398535, 'EX_cpd00028_e': 0.6536739717398535, 'EX_cpd00066_e': 0.6536739717398535, 'EX_cpd01188_e': 0.6536739717398535, 'EX_cpd00357_e': 0.6536739717402804, 'EX_cpd00136_e': 0.6536739717398535, '

In [21]:
path_to_pickle = "/Users/basazinbelhu/probannopy/iterative_pfba/data/medusa_gapfilled_model_pickle/Noise_0.25/"
pkl_files = [pos_pkl for pos_pkl in os.listdir(path_to_pickle) if pos_pkl.endswith('.pkl')]

In [22]:
data_val(path_to_pickle)

Clostridium saccharolyticum WM1.pkl
       rxn_id      formula                                               name
0  cpd00443_c      C7H6NO2                                               ABEE
1  cpd02920_c   C7H9N5O8P2  2-Amino-4-hydroxy-6-hydroxymethyl-7-8-dihydrop...
4  cpd00683_c   C14H13N6O3                                    Dihydropteroate
5  cpd00114_c  C10H11N4O8P                                                IMP
6  cpd00103_c    C5H9O14P3                                               PRPP
1325


2021-03-17 11:13:21.433 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:21.442 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:21.452 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:21.499 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:21.520 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:21.545 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:21.568 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:21.589 Compartment `e` sounds like an external compartment. Using this one without coun

2021-03-17 11:13:23.507 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:23.529 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:23.550 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:23.575 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:23.593 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:23.611 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:23.628 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:23.646 Compartment `e` sounds like an external compartment. Using this one without coun

2021-03-17 11:13:25.517 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:25.540 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:25.557 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:25.576 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:25.599 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:25.617 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:25.636 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:25.656 Compartment `e` sounds like an external compartment. Using this one without coun

2021-03-17 11:13:27.839 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:27.868 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:27.892 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:27.918 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:27.946 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:27.966 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:27.988 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:13:28.008 Compartment `e` sounds like an external compartment. Using this one without coun

...

2021-03-17 11:20:40.305 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:40.325 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:40.343 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:40.360 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:40.379 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:40.397 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:40.414 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:40.433 Compartment `e` sounds like an external compartment. Using this one without coun

2021-03-17 11:20:43.077 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:43.099 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:43.119 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:43.137 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:43.157 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:43.177 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:43.197 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:43.215 Compartment `e` sounds like an external compartment. Using this one without coun

2021-03-17 11:20:45.169 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:45.186 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:45.204 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:45.223 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:45.241 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:45.258 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:45.276 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:45.295 Compartment `e` sounds like an external compartment. Using this one without coun

2021-03-17 11:20:47.769 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:47.789 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:47.807 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:47.828 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:47.846 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:47.863 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:47.882 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:47.900 Compartment `e` sounds like an external compartment. Using this one without coun

2021-03-17 11:20:49.635 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:49.652 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:49.668 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:49.686 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:49.702 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:49.718 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:49.735 Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions
2021-03-17 11:20:49.752 Compartment `e` sounds like an external compartment. Using this one without coun

In [23]:
styled_data[5]

,EX_cpd00635_e,EX_cpd00023_e,EX_cpd00224_e,EX_cpd11595_e,EX_cpd01217_e,EX_cpd00027_e,EX_cpd01171_e,EX_cpd00154_e,EX_cpd08306_e,EX_cpd00164_e,EX_cpd00309_e,EX_cpd00280_e,EX_cpd00108_e,EX_cpd00159_e,EX_cpd00540_e,EX_cpd11597_e,EX_cpd08305_e,EX_cpd00396_e,EX_cpd00039_e,EX_cpd00011_e,EX_cpd00322_e,EX_cpd00111_e,EX_cpd00028_e,EX_cpd00066_e,EX_cpd01188_e,EX_cpd00357_e,EX_cpd00136_e,EX_cpd03847_e,EX_cpd01080_e,EX_cpd03422_e,EX_cpd00010_e,EX_cpd00264_e,EX_cpd11606_e,EX_cpd00069_e,EX_cpd00060_e,EX_cpd00305_e
B. vulgatus_gapfilled_0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012705,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010999,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001399,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023741,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105870,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007142,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B. vulgatus_gapfilled_9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006098,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
styled_data[5].columns

Index(['EX_cpd00635_e', 'EX_cpd00023_e', 'EX_cpd00224_e', 'EX_cpd11595_e',
       'EX_cpd01217_e', 'EX_cpd00027_e', 'EX_cpd01171_e', 'EX_cpd00154_e',
       'EX_cpd08306_e', 'EX_cpd00164_e', 'EX_cpd00309_e', 'EX_cpd00280_e',
       'EX_cpd00108_e', 'EX_cpd00159_e', 'EX_cpd00540_e', 'EX_cpd11597_e',
       'EX_cpd08305_e', 'EX_cpd00396_e', 'EX_cpd00039_e', 'EX_cpd00011_e',
       'EX_cpd00322_e', 'EX_cpd00111_e', 'EX_cpd00028_e', 'EX_cpd00066_e',
       'EX_cpd01188_e', 'EX_cpd00357_e', 'EX_cpd00136_e', 'EX_cpd03847_e',
       'EX_cpd01080_e', 'EX_cpd03422_e', 'EX_cpd00010_e', 'EX_cpd00264_e',
       'EX_cpd11606_e', 'EX_cpd00069_e', 'EX_cpd00060_e', 'EX_cpd00305_e'],
      dtype='object')

In [19]:
# !pip install streamlit

In [18]:
styled_data[5]

IndexError: list index out of range

In [ ]:
for i in range(styled_data):
    lit =[ f for f in styled_data[i].columns]
    